In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Import plotting modules
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, VotingRegressor

from sklearn.neural_network import MLPRegressor


from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import metrics

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) 
# that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/us-counties-covid-19-dataset/us-counties.csv')
df['date'] = pd.to_datetime(df.date)
df['date1'] = df['date'].dt.strftime("%Y%m%d").astype(int)
df2 = df

df2a = df2[df2['date'] == df['date'].max()]
df2a = df2a.groupby(['state','county'])[['deaths','cases']].sum()
df2ab = df2a.sort_values(['deaths','state'])
df2ac = df2a.sort_values(['cases','state'])
df2ab = df2ab.reset_index() ## 30 Most Death Counties
df2ac = df2ac.reset_index() ## 30 Most Cases Counties
df2ab = df2ab.tail(30)
df2ac = df2ac.tail(30)


df_a = df2.groupby(['state','county'])[['deaths','cases']].sum()
df_b = df_a.sort_values(['deaths','state'])
df_c = df_a.sort_values(['cases','state'])

df_b = df_b.tail(30) ## 30 Most Death Counties
df_c = df_c.tail(30) ## 30 Most Cases Counties



df_b = df_b.reset_index() ## 30 Most Death Counties
df_c = df_c.reset_index() ## 30 Most Cases Counties


df_aa = (df_b['state'].unique()) ## 30 Most Death Counties
df_ab = (df_c['state'].unique()) ## 30 Most Cases Counties

d1 = {} ## 30 Most Death Counties
d2 = {} ## 30 Most Cases Counties
for a in df_aa:
    d1[a] = []
    b = df_b.loc[(df_b['state'] == a),'county']
    for aa in b:
        d1[a].append(aa)

for a in df_ab:
    d2[a] = []
    b = df_c.loc[(df_c['state'] == a),'county']
    for aa in b:
        d2[a].append(aa)
        

## Latest US Deaths & Cases

In [ ]:
df_all = df
df_all = df_all.groupby(['date'])[['deaths','cases']].sum()
df_alll = df_all

df_all = df_all.reset_index()
df_al = df_all[df_all['date'] == df_all['date'].max()]
df_al

In [ ]:
df_all_1 = df_alll.diff(axis=0)
df_all_1 = df_all_1.reset_index()
df_all_1 = df_all_1.reset_index()
print(df_all_1.tail())


In [ ]:
titleaa = 'US' 
titlea = 'Deaths vs Date'
titleb = 'Cases vs Date'
# Set default Seaborn style
fig, axes = plt.subplots(2,1 , figsize=(15, 5), sharex= True)
fig.suptitle(titleaa)
sns.regplot(ax=axes[0], x = 'index', y= 'deaths', data = df_all_1)
axes[0].set_title(titlea)
sns.regplot(ax=axes[1], x = 'index', y= 'cases', data = df_all_1)
axes[1].set_title(titleb)
plt.show()

## Regression Model on US Deaths & Cases

In [ ]:
x = df_all_1.iloc[1:,0].values.reshape(-1,1)
y = df_all_1.iloc[1:,2].values.reshape(-1,1)
y2 = df_all_1.iloc[1:,3].values.reshape(-1,1)

max_x = x.max()
x_future = max_x + 10
x2 = np.arange(max_x,x_future).reshape(-1,1)

# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size = 0.3, random_state=42)

# Create training and test sets
X_train2, X_test2, y_train2, y_test2 = train_test_split(x, y2,test_size = 0.3, random_state=42)

**Ensamble Model (LR, GBR, RFR)**

In [ ]:
reg1 = GradientBoostingRegressor(random_state=1, n_estimators=10)
reg2 = RandomForestRegressor(random_state=1, n_estimators=10)
reg3 = LinearRegression()
ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3)])

ereg = ereg.fit(X_train, y_train)
y_pred_death = ereg.predict(X_test)
x_22 = ereg.predict(x2)
x_22 = np.rint(x_22)
print('################# Deaths in next 10 Days #################')
print(x_22)

print('#########Deaths Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_death))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_death))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_death)))
print('R^2:', np.sqrt(metrics.r2_score(y_test, y_pred_death)))

ereg = ereg.fit(X_train2, y_train2)
y_pred_cases = ereg.predict(X_test2)
x_22 = ereg.predict(x2)
x_22 = np.rint(x_22)
print('################# Cases in next 10 Days #################')
print(x_22)
print('#########Cases Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test2, y_pred_cases))
print('Mean Squared Error:', metrics.mean_squared_error(y_test2, y_pred_cases))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test2, y_pred_cases)))
print('R^2:', np.sqrt(metrics.r2_score(y_test2, y_pred_cases)))

**Linear Regression**

In [ ]:
l1 = LinearRegression()
l1.fit(X_train,y_train)
y_pred_death = l1.predict(X_test)

print('################# Deaths in next 10 Days #################')
x_22 = l1.predict(x2)
x_22 = np.rint(x_22)
print(x_22)

l1.fit(X_train2,y_train2)
y_pred_cases = l1.predict(X_test2)
print('################# Cases in next 10 Days #################')
x_22 = l1.predict(x2)
x_22 = np.rint(x_22)
print(x_22)


plt.scatter(x,y)
plt.plot(X_test,y_pred_death,color='red')
plt.title('Linear Regression')
plt.xlabel('Days')
plt.ylabel('Deaths')
plt.show()
plt.scatter(x,y2)
plt.plot(X_test2,y_pred_cases,color='red')
plt.title('Linear Regression')
plt.xlabel('Days')
plt.ylabel('Cases')
plt.show()


print('#########Deaths Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_death))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_death))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_death)))
print('R^2:', np.sqrt(metrics.r2_score(y_test, y_pred_death)))

print('#########Cases Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test2, y_pred_cases))
print('Mean Squared Error:', metrics.mean_squared_error(y_test2, y_pred_cases))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test2, y_pred_cases)))
print('R^2:', np.sqrt(metrics.r2_score(y_test2, y_pred_cases)))

In [ ]:
max_x = x.max()
x_future = max_x + 10
x2 = np.arange(max_x,x_future).reshape(-1,1)

poly_reg = PolynomialFeatures(degree=4)

x = df_all_1.iloc[1:,0].values.reshape(-1,1)
poly_x = poly_reg.fit_transform(x)

y = df_all_1.iloc[1:,2].values.reshape(-1,1)
y2 = df_all_1.iloc[1:,3].values.reshape(-1,1)

l1 = LinearRegression()
l1.fit(poly_x,y)
y_pred_death = l1.predict(poly_x)

print('################# Deaths in next 10 Days #################')
x_22 = l1.predict(poly_reg.fit_transform(x2))
x_22 = np.rint(x_22)
print(x_22)

l1.fit(poly_x,y2)
y_pred_cases = l1.predict(poly_x)
print('################# Cases in next 10 Days #################')
x_22 = l1.predict(poly_reg.fit_transform(x2))
x_22 = np.rint(x_22)
print(x_22)


plt.scatter(x,y)
plt.plot(x,y_pred_death,color='red')
plt.title('4th Polynomial Regression')
plt.xlabel('Days')
plt.ylabel('Deaths')
plt.show()
plt.scatter(x,y2)
plt.plot(x,y_pred_cases,color='red')
plt.title('4th Polynomial Regression')
plt.xlabel('Days')
plt.ylabel('Cases')
plt.show()

print('#########Deaths Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y, y_pred_death))
print('Mean Squared Error:', metrics.mean_squared_error(y, y_pred_death))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y, y_pred_death)))
print('R^2:', np.sqrt(metrics.r2_score(y, y_pred_death)))

print('#########Cases Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y2, y_pred_cases))
print('Mean Squared Error:', metrics.mean_squared_error(y2, y_pred_cases))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y2, y_pred_cases)))
print('R^2:', np.sqrt(metrics.r2_score(y2, y_pred_cases)))

In [ ]:
max_x = x.max()
x_future = max_x + 10
x2 = np.arange(max_x,x_future).reshape(-1,1)

poly_reg = PolynomialFeatures(degree=8)

x = df_all_1.iloc[1:,0].values.reshape(-1,1)
poly_x = poly_reg.fit_transform(x)

y = df_all_1.iloc[1:,2].values.reshape(-1,1)
y2 = df_all_1.iloc[1:,3].values.reshape(-1,1)

l1 = LinearRegression()
l1.fit(poly_x,y)
y_pred_death = l1.predict(poly_x)

print('################# Deaths in next 10 Days #################')
x_22 = l1.predict(poly_reg.fit_transform(x2))
x_22 = np.rint(x_22)
print(x_22)

l1.fit(poly_x,y2)
y_pred_cases = l1.predict(poly_x)
print('################# Cases in next 10 Days #################')
x_22 = l1.predict(poly_reg.fit_transform(x2))
x_22 = np.rint(x_22)
print(x_22)


plt.scatter(x,y)
plt.plot(x,y_pred_death,color='red')
plt.title('8th Polynomial Regression')
plt.xlabel('Days')
plt.ylabel('Deaths')
plt.show()
plt.scatter(x,y2)
plt.plot(x,y_pred_cases,color='red')
plt.title('8th Polynomial Regression')
plt.xlabel('Days')
plt.ylabel('Cases')
plt.show()

print('#########Deaths Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y, y_pred_death))
print('Mean Squared Error:', metrics.mean_squared_error(y, y_pred_death))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y, y_pred_death)))
print('R^2:', np.sqrt(metrics.r2_score(y, y_pred_death)))

print('#########Cases Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y2, y_pred_cases))
print('Mean Squared Error:', metrics.mean_squared_error(y2, y_pred_cases))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y2, y_pred_cases)))
print('R^2:', np.sqrt(metrics.r2_score(y2, y_pred_cases)))

**Multi Layer Perceptron**

In [ ]:
x = df_all_1.iloc[1:,0].values.reshape(-1,1)
y = df_all_1.iloc[1:,2].values.reshape(-1,1)
y2 = df_all_1.iloc[1:,3].values.reshape(-1,1)

max_x = x.max()
x_future = max_x + 10
x2 = np.arange(max_x,x_future).reshape(-1,1)


regr = MLPRegressor(random_state=1, hidden_layer_sizes=(100, 100,100,100, 100,100), max_iter=500).fit(x, y)
y_pred_death = regr.predict(x)

print('################# Deaths in next 10 Days #################')
x_22 = regr.predict(x2)
x_22 = np.rint(x_22)
print(x_22)


regr = MLPRegressor(random_state=1, hidden_layer_sizes=(100, 100,100,100, 100,100), max_iter=500).fit(x, y2)
y_pred_cases = regr.predict(x)
print('################# Cases in next 10 Days #################')
x_22 = regr.predict(x2)
x_22 = np.rint(x_22)
print(x_22)

plt.scatter(x,y)
plt.plot(x,y_pred_death,color='red')
plt.title('MLPRegressor')
plt.xlabel('Days')
plt.ylabel('Deaths')
plt.show()
plt.scatter(x,y2)
plt.plot(x,y_pred_cases,color='red')
plt.title('MLPRegressor')
plt.xlabel('Days')
plt.ylabel('Cases')
plt.show()

print('#########Deaths Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y, y_pred_death))
print('Mean Squared Error:', metrics.mean_squared_error(y, y_pred_death))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y, y_pred_death)))
print('R^2:', np.sqrt(metrics.r2_score(y, y_pred_death)))

print('#########Cases Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y2, y_pred_cases))
print('Mean Squared Error:', metrics.mean_squared_error(y2, y_pred_cases))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y2, y_pred_cases)))
print('R^2:', np.sqrt(metrics.r2_score(y2, y_pred_cases)))

In [ ]:
poly_reg = PolynomialFeatures(degree=3)

x = df_all_1.iloc[1:,0].values.reshape(-1,1)
y = df_all_1.iloc[1:,2].values.reshape(-1,1)
y2 = df_all_1.iloc[1:,3].values.reshape(-1,1)



poly_x = poly_reg.fit_transform(x)

regr = MLPRegressor(random_state=1, hidden_layer_sizes=(25, 57,57,34), max_iter=500).fit(poly_x, y)
y_pred_death = regr.predict(poly_x)

print('################# Deaths in next 10 Days #################')
x_22 = regr.predict(poly_reg.fit_transform(x2))
x_22 = np.rint(x_22)
print(x_22)


regr = MLPRegressor(random_state=1, hidden_layer_sizes=(100, 100), max_iter=500).fit(poly_x, y2)
y_pred_cases = regr.predict(poly_x)
print('################# Cases in next 10 Days #################')
x_22 = regr.predict(poly_reg.fit_transform(x2))
x_22 = np.rint(x_22)
print(x_22)

plt.scatter(x,y)
plt.plot(x,y_pred_death,color='red')
plt.title('MLPRegressor 3rd degree')
plt.xlabel('Days')
plt.ylabel('Deaths')
plt.show()
plt.scatter(x,y2)
plt.plot(x,y_pred_cases,color='red')
plt.title('MLPRegressor 3rd degree')
plt.xlabel('Days')
plt.ylabel('Cases')
plt.show()

print('#########Deaths Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y, y_pred_death))
print('Mean Squared Error:', metrics.mean_squared_error(y, y_pred_death))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y, y_pred_death)))
print('R^2:', np.sqrt(metrics.r2_score(y, y_pred_death)))

print('#########Cases Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y2, y_pred_cases))
print('Mean Squared Error:', metrics.mean_squared_error(y2, y_pred_cases))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y2, y_pred_cases)))
print('R^2:', np.sqrt(metrics.r2_score(y2, y_pred_cases)))

**Random Foreset**

In [ ]:
# Instantiate dt
dt = DecisionTreeRegressor(max_depth=6,
            min_samples_leaf=0.009,
            random_state=3)

# Fit dt to the training set
dt.fit(X_train, y_train)
y_pred_death = dt.predict(X_test)
y_pred_1 = dt.predict(X_test)
print('################# Deaths in next 10 Days #################')
x_22 = dt.predict(x2)
x_22 = np.rint(x_22)
print(x_22)

dt.fit(X_train2,y_train2)
y_pred_cases = dt.predict(X_test2)
print('################# Cases in next 10 Days #################')
x_22 = dt.predict(x2)
x_22 = np.rint(x_22)
print(x_22)

print('#########Deaths Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_death))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_death))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_death)))
print('R^2:', np.sqrt(metrics.r2_score(y_test, y_pred_death)))

print('#########Cases Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test2, y_pred_cases))
print('Mean Squared Error:', metrics.mean_squared_error(y_test2, y_pred_cases))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test2, y_pred_cases)))
print('R^2:', np.sqrt(metrics.r2_score(y_test2, y_pred_cases)))

In [ ]:
df_all = df
df_all = df_all.groupby(['date'])[['deaths','cases']].sum()
df_alll = df_all

df_alll = df_alll.diff(axis=0)

df_alll1 = df_alll.reset_index()

mean1 = df_alll1['deaths'].mean()
mean2 = df_alll1['cases'].mean()


a = df_alll.idxmax()


titleaa = 'US' 
titlea = 'Deaths vs Date'
titleb = 'Cases vs Date'
# Set default Seaborn style
fig, axes = plt.subplots(2,1 , figsize=(15, 5), sharex=True)
fig.suptitle(titleaa)
sns.lineplot(ax=axes[0], x = 'date', y= 'deaths', data = df_alll1)
axes[0].set_title(titlea)
sns.lineplot(ax=axes[1], x = 'date', y= 'cases', data = df_alll1)
axes[1].set_title(titleb)

axes[0].axhline(mean1, color='r', linestyle='--', label = 'Mean Deaths')
axes[1].axhline(mean2, color='b', linestyle='--', label = 'Mean Cases')

axes[0].axvline(a[0], color='B', linestyle='-', label = 'Date of max Deaths')
axes[1].axvline(a[1], color='R', linestyle='-', label = 'Date of max Cases')

# Customize the labels and limits
axes[0].legend()
axes[1].legend()
plt.xticks(rotation=45)
# Show TimeSeries
plt.show()

## Regression Model on Cook County IL Deaths & Cases

In [ ]:
df_all_12 = df
df_all_12 = df_all_12[df_all_12['state'] == 'Illinois']
df_all_12 = df_all_12[df_all_12['county'] == 'Cook']
df_all_12 = df_all_12.groupby(['date'])[['deaths','cases']].sum()
df_all_123 = df_all_12

df_all_123 = df_all_123.diff(axis = 0)
df_all_123 = df_all_123.reset_index()
df_all_123 = df_all_123.reset_index()
date = df_all_123[df_all_123['date'] == df_all_123['date'].max()]

print(df_all_123.tail())
print(date)

In [ ]:
x = df_all_123.iloc[1:,0].values.reshape(-1,1)
y = df_all_123.iloc[1:,2].values.reshape(-1,1)
y2 = df_all_123.iloc[1:,3].values.reshape(-1,1)

max_x = x.max()
x_future = max_x + 10
x2 = np.arange(max_x,x_future).reshape(-1,1)

l1 = LinearRegression()
l1.fit(x,y)
y_pred_death = l1.predict(x)

print('################# Deaths in next 10 Days #################')
x_22 = l1.predict(x2)
x_22 = np.rint(x_22)
print(x_22)

l1.fit(x,y2)
y_pred_cases = l1.predict(x)
print('################# Cases in next 10 Days #################')
x_22 = l1.predict(x2)
x_22 = np.rint(x_22)
print(x_22)


plt.scatter(x,y)
plt.plot(x,y_pred_death,color='red')
plt.title('Linear Regression')
plt.xlabel('Days')
plt.ylabel('Deaths')
plt.show()
plt.scatter(x,y2)
plt.plot(x,y_pred_cases,color='red')
plt.title('Linear Regression')
plt.xlabel('Days')
plt.ylabel('Cases')
plt.show()


print('#########Deaths Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y, y_pred_death))
print('Mean Squared Error:', metrics.mean_squared_error(y, y_pred_death))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y, y_pred_death)))
print('R^2:', np.sqrt(metrics.r2_score(y, y_pred_death)))

print('#########Cases Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y2, y_pred_cases))
print('Mean Squared Error:', metrics.mean_squared_error(y2, y_pred_cases))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y2, y_pred_cases)))
print('R^2:', np.sqrt(metrics.r2_score(y2, y_pred_cases)))

In [ ]:
max_x = x.max()
x_future = max_x + 10
x2 = np.arange(max_x,x_future).reshape(-1,1)

poly_reg = PolynomialFeatures(degree=4)

x = df_all_123.iloc[1:,0].values.reshape(-1,1)
poly_x = poly_reg.fit_transform(x)

y = df_all_123.iloc[1:,2].values.reshape(-1,1)
y2 = df_all_123.iloc[1:,3].values.reshape(-1,1)

l1 = LinearRegression()
l1.fit(poly_x,y)
y_pred_death = l1.predict(poly_x)

print('################# Deaths in next 10 Days #################')
x_22 = l1.predict(poly_reg.fit_transform(x2))
x_22 = np.rint(x_22)
print(x_22)

l1.fit(poly_x,y2)
y_pred_cases = l1.predict(poly_x)
print('################# Cases in next 10 Days #################')
x_22 = l1.predict(poly_reg.fit_transform(x2))
x_22 = np.rint(x_22)
print(x_22)


plt.scatter(x,y)
plt.plot(x,y_pred_death,color='red')
plt.title('4th Polynomial Regression')
plt.xlabel('Days')
plt.ylabel('Deaths')
plt.show()
plt.scatter(x,y2)
plt.plot(x,y_pred_cases,color='red')
plt.title('4th Polynomial Regression')
plt.xlabel('Days')
plt.ylabel('Cases')
plt.show()

print('#########Deaths Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y, y_pred_death))
print('Mean Squared Error:', metrics.mean_squared_error(y, y_pred_death))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y, y_pred_death)))
print('R^2:', np.sqrt(metrics.r2_score(y, y_pred_death)))

print('#########Cases Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y2, y_pred_cases))
print('Mean Squared Error:', metrics.mean_squared_error(y2, y_pred_cases))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y2, y_pred_cases)))
print('R^2:', np.sqrt(metrics.r2_score(y2, y_pred_cases)))

In [ ]:
max_x = x.max()
x_future = max_x + 10
x2 = np.arange(max_x,x_future).reshape(-1,1)

poly_reg = PolynomialFeatures(degree=8)

x = df_all_123.iloc[1:,0].values.reshape(-1,1)
poly_x = poly_reg.fit_transform(x)

y = df_all_123.iloc[1:,2].values.reshape(-1,1)
y2 = df_all_123.iloc[1:,3].values.reshape(-1,1)

l1 = LinearRegression()
l1.fit(poly_x,y)
y_pred_death = l1.predict(poly_x)

print('################# Deaths in next 10 Days #################')
x_22 = l1.predict(poly_reg.fit_transform(x2))
x_22 = np.rint(x_22)
print(x_22)

l1.fit(poly_x,y2)
y_pred_cases = l1.predict(poly_x)
print('################# Cases in next 10 Days #################')
x_22 = l1.predict(poly_reg.fit_transform(x2))
x_22 = np.rint(x_22)
print(x_22)


plt.scatter(x,y)
plt.plot(x,y_pred_death,color='red')
plt.title('8th Polynomial Regression')
plt.xlabel('Days')
plt.ylabel('Deaths')
plt.show()
plt.scatter(x,y2)
plt.plot(x,y_pred_cases,color='red')
plt.title('8th Polynomial Regression')
plt.xlabel('Days')
plt.ylabel('Cases')
plt.show()

print('#########Deaths Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y, y_pred_death))
print('Mean Squared Error:', metrics.mean_squared_error(y, y_pred_death))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y, y_pred_death)))
print('R^2:', np.sqrt(metrics.r2_score(y, y_pred_death)))

print('#########Cases Model###################')
print('Mean Absolute Error:', metrics.mean_absolute_error(y2, y_pred_cases))
print('Mean Squared Error:', metrics.mean_squared_error(y2, y_pred_cases))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y2, y_pred_cases)))
print('R^2:', np.sqrt(metrics.r2_score(y2, y_pred_cases)))


## Top 30 Counties with Most Covid Cases

In [ ]:
df2ac.sort_values('cases')

In [ ]:
for key, values in d2.items():
    df22 = df2[df2['state'] == key]
    for a in values:
        df33 = df22[df22['county'] == a]
                
        df33 = df33.groupby(['state','county','date'])[['deaths','cases']].sum()
        df33 = df33.diff(axis=0)
        df333 = df33[df33['deaths'] >= 0]
        df333 = df33[df33['cases'] >= 0]
        
        aa = df33.idxmax()
        mean1 = df33['deaths'].mean()
        mean2 = df33['cases'].mean()
        dff333 = df333.reset_index()


        titleaa = 'State - ' + key +' || County - ' + a 
        titlea = 'Deaths vs Date'
        titleb = 'Case vs Date'
        # Set default Seaborn style
        fig, axes = plt.subplots(2,1 , figsize=(15, 5), sharex=True)
        fig.suptitle(titleaa)
        sns.lineplot(ax=axes[0], x = 'date', y= 'deaths', data = dff333)
        axes[0].set_title(titlea)
        sns.lineplot(ax=axes[1], x = 'date', y= 'cases', data = dff333)
        axes[1].set_title(titleb)
        
        axes[0].axhline(mean1, color='r', linestyle='--', label = 'Mean Deaths')
        axes[1].axhline(mean2, color='b', linestyle='--', label = 'Mean Cases')
        
        axes[0].axvline(aa[0][2], color='B', linestyle='-', label = 'Date of max Deaths')
        axes[1].axvline(aa[1][2], color='R', linestyle='-', label = 'Date of max Cases')
        
        axes[0].legend()
        axes[1].legend()
        # Customize the labels and limits
        plt.xticks(rotation=45)
        # Show TimeSeries
        plt.show()

## Top 30 Counties with Most Deaths

In [ ]:
df2ab.sort_values('deaths')

In [ ]:
for key, values in d1.items():
    df22 = df2[df2['state'] == key]
    for a in values:
        df33 = df22[df22['county'] == a]
        df33 = df33.groupby(['state','county','date'])[['deaths','cases']].sum()
        df33 = df33.diff(axis=0)
        df333 = df33[df33['deaths'] >= 0]
        df333 = df33[df33['cases'] >= 0]
        dff333 = df333.reset_index()


        titleaa = 'State - ' + key +' || County - ' + a 
        titlea = 'Deaths vs Date'
        titleb = 'Case vs Date'
        # Set default Seaborn style
        fig, axes = plt.subplots(2,1 , figsize=(15, 5), sharex=True)
        fig.suptitle(titleaa)
        sns.lineplot(ax=axes[0], x = 'date', y= 'deaths', data = dff333)
        axes[0].set_title(titlea)
        sns.lineplot(ax=axes[1], x = 'date', y= 'cases', data = dff333)
        axes[1].set_title(titleb)
        # Customize the labels and limits
        plt.xticks(rotation=45)
        # Show TimeSeries
        plt.show()

In [ ]:
df222 = df

df222_a = df222.groupby(['state'])[['deaths','cases']].sum()
df222_ab = df222_a.sort_values('deaths')
df222_ac = df222_a.sort_values('cases')
df222_ab = df222_ab.reset_index()
df222_ac = df222_ac.reset_index()

df222_ab = df222_ab.tail(15)
df222_ac = df222_ac.tail(15)

titlea = 'Deaths vs Date'
titleb = 'Case vs Date'

# Set default Seaborn style
fig, axes = plt.subplots(2,1 , figsize=(15, 17), sharex = True)
sns.regplot(ax=axes[0], x = 'cases', y= 'deaths', data = df222_ab, label = 'state')
axes[0].legend()

sns.regplot(ax=axes[1], x = 'cases', y= 'deaths', data = df222_ac, label = 'state')
axes[1].legend()

def label_point(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x']+.1, point['y'], str(point['val']))

    

label_point(df222_ab.cases, df222_ab.deaths, df222_ab.state, axes[0])
label_point(df222_ac.cases, df222_ac.deaths, df222_ac.state, axes[1])

fig.suptitle('Deaths vs Cases by State')
axes[0].set_title('Top 15 States with most Deaths')
axes[1].set_title('Top 15 States with most Cases')
plt.show()

In [ ]:
df_state = pd.read_csv('../input/state-data1/csvData (1).csv')
df222_a1 = df222_a.reset_index()
df222_a1 = df222_a1.rename(columns={"state": "State"})
result = pd.merge(df222_a1, df_state, how='inner', on='State')

result1 = result.sort_values('deaths')
result2 = result.sort_values('cases')

result1 = result1.tail(15)
result2 = result2.tail(15)

# Set default Seaborn style
fig, axes = plt.subplots(2,1 , figsize=(15, 15), sharex = True)
sns.regplot(ax=axes[0], x = 'Pop', y= 'cases', data = result1, label = 'state')
axes[0].legend()

sns.regplot(ax=axes[1], x = 'Pop', y= 'deaths', data = result1, label = 'state')
axes[1].legend()

def label_point(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x']+.1, point['y'], str(point['val']))

    

label_point(result1.Pop, result1.cases, result1.State, axes[0])
label_point(result1.Pop, result1.deaths, result1.State, axes[1])

fig.suptitle('Population vs Deaths/Cases by State')
axes[0].set_title('Top 15 States with most Deaths')
axes[1].set_title('Top 15 States with most Cases')
plt.show()

In [ ]:
result['Crude death rate'] = result.deaths / result.Pop * 100
result1['Crude death rate'] = result1.deaths / result1.Pop * 100
result2['Crude death rate'] = result2.deaths / result2.Pop * 100

result['Point prevalence'] = result.cases / result.Pop
result1['Point prevalence'] = result1.cases / result1.Pop
result2['Point prevalence'] = result2.cases / result2.Pop

result['case fatality proportion'] = result.deaths / result.cases
result1['case fatality proportion'] = result1.deaths / result1.cases
result2['case fatality proportion'] = result2.deaths / result2.cases

result1 = result1.sort_values('Crude death rate')
result2 = result2.sort_values('Crude death rate')


# Set default Seaborn style
fig, axes = plt.subplots(2,1 , figsize=(15, 15), sharex = True)
sns.regplot(ax=axes[0], x = 'Point prevalence', y= 'Crude death rate', data = result1, label = 'state')
axes[0].legend()

sns.regplot(ax=axes[1], x = 'Point prevalence', y= 'Crude death rate', data = result2, label = 'state')
axes[1].legend()

def label_point(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x']+.05, point['y'], str(point['val']))

    

label_point(result1['Point prevalence'], result1['Crude death rate'], result1.State, axes[0])
label_point(result2['Point prevalence'], result2['Crude death rate'], result2.State, axes[1])

fig.suptitle('Epi Measures')
axes[0].set_title('Top 15 States with most Deaths')
axes[1].set_title('Top 15 States with most Cases')
plt.show()

In [ ]:
plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(result.corr(), vmin=-0.95, vmax=0.95, annot=True, cmap='BrBG')
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=12);
plt.show()

In [ ]:
# Set default Seaborn style
fig, axes = plt.subplots(2,1 , figsize=(15, 15), sharex = True)
sns.regplot(ax=axes[0], x = 'Poverty', y= 'deaths', data = result, label = 'state')
axes[0].legend()

sns.regplot(ax=axes[1], x = 'Poverty', y= 'cases', data = result, label = 'state')
axes[1].legend()

def label_point(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x']+.05, point['y'], str(point['val']))

    

label_point(result['Poverty'], result['deaths'], result.State, axes[0])
label_point(result['Poverty'], result['cases'], result.State, axes[1])

fig.suptitle('Epi Measures')
axes[0].set_title('Poverty vs Deaths')
axes[1].set_title('Poverty vs Cases')
plt.show()